# Wine Quality using Artificial Neural Network (ANN)

In [ ]:
import numpy as np
import pandas as pd
import math

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Prepare Data

In [ ]:
path = '../input/wine-quality/winequalityN.csv'
df = pd.read_csv(path)
df.head()

In [ ]:
labels = np.unique(df['type'].values)
idx_to_labels = { k:v for k,v in enumerate(labels) }
labels_to_idx = { v:k for k,v in enumerate(labels) }

In [ ]:
labels = df.replace(labels_to_idx)['type'].values
df = df.drop(columns=['type'])

# one hot encoding
labels = np.eye(len(idx_to_labels))[labels]

In [ ]:
# normalized data
df = (df-df.mean())/df.std()

# replace NaN with Standard Deviation
df = df.fillna(df.std())

df.head()

In [ ]:
features = df.values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.33, random_state=42)

# Parameters

In [ ]:
gamma = {}
gamma["ndims"] = features.shape[1]
gamma["nclasses"] = len(idx_to_labels.values())

# Generate Weights

In [ ]:
np.random.seed(1337)
def generate_weights(gamma):
    '''
        Generate Weights and use Xavier Initiation
    '''
    scale = 1/max(1., (2+2)/2.)
    limit = math.sqrt(3.0 * scale)

    gamma['w0'] = np.random.uniform(-limit, limit, size=(gamma['ndims'], gamma['ndims']))
    gamma['w1'] = np.random.uniform(-limit, limit, size=(gamma['ndims'], gamma['nclasses']))
    
    return gamma

In [ ]:
gamma = generate_weights(gamma)
print('w0 shape:', gamma['w0'].shape, ' - w1 shape:', gamma['w1'].shape)

# Activation Function and Derivative

In [ ]:
def sigmoid(x):
    return 1. / (1. + np.exp(-x))

def dsigmoid(x):
    return x * (1. - x)

# Training

In [ ]:
def loss(y, y_hat):
    '''
        Addition of all Squared Mean Errors
    '''
    return np.sum(np.mean(np.square(np.subtract(y, y_hat)), axis=0))

In [ ]:
def forward(X, gamma):
    '''
        Forward Propagation
    '''
    l0 = X
    l1 = sigmoid(np.dot(l0, gamma['w0']))
    l2 = sigmoid(np.dot(l1, gamma['w1']))
    
    return l0, l1, l2

In [ ]:
def backward(y, theta, gamma, lr):
    '''
        Backward Propagation
    '''
    l0, l1, l2 = theta
    
    l2_error = y - l2
    l2_delta = l2_error * dsigmoid(l2)
    l1_error = l2_delta.dot(gamma['w1'].T)
    l1_delta = l1_error * dsigmoid(l1)
    
    # update using SGD
    gamma['w0'] += lr * l0.T.dot(l1_delta)
    gamma['w1'] += lr * l1.T.dot(l2_delta)
    
    return gamma

In [ ]:
def train(X, y, gamma, iterations=60, lr=0.01):
    '''
        Function to Train Dataset
    '''
    errors = []
    for i in range(iterations):
        # forward propagation
        theta = forward(X, gamma)
        
        e = loss(theta[-1], y)
        if(i % 4 == 0):
            print('I:{0:4d}, --  Mean Error:{1:1.4f}'.format(i, np.mean(e)))
        errors.append(e)

        # backward propagation
        gamma = backward(y, theta, gamma, lr)
            
    return gamma, errors

In [ ]:
gamma, errors = train(X_train, y_train, gamma)

# Plot Error Lost

In [ ]:
plt.plot(errors)

# Accuracy

In [ ]:
def accuracy(y, gamma):
    '''
    Function to calculate accuracy
    '''
    acc_y = []
    for x in X_test:
        y = np.argmax(forward(x.reshape(1, 12), gamma)[-1])
        y = np.eye(gamma["nclasses"])[y]
        acc_y.append(y)

    acc_y = np.array(acc_y)
    wrong = len(np.where(np.equal(y_test, acc_y).astype(int) == 0)[0])
    return 1 - (len(y) / wrong)

In [ ]:
 print('Accuracy:{0:3d}%'.format(int(accuracy(y_train, gamma) * 100)))